In [3]:
%load_ext autoreload
%autoreload 2
import multiprocessing
import os
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [4]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-21T03:30:00"
event_start = "2010-01-20"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / "runs/gridrad_demo"
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)
lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)
gridrad_options = data_options["gridrad"]

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options = None

2024-10-18 17:00:41,294 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/data.yml
2024-10-18 17:00:41,310 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-18 17:00:41,311 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/grid.yml


In [5]:
times = data.utils.generate_times(data_options["gridrad"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_parent
)

2024-10-18 17:00:41,404 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo.
2024-10-18 17:00:41,405 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-18 17:00:41,480 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-10-18 17:00:41,483 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-10-18 17:00:41,485 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-18 17:00:41,486 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-18 17:00:41,665 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-18 17:00:42,903 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-18 17:00:42,950 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-18 17:00:43,211 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-18 

KeyboardInterrupt: 

In [6]:
import logging

def get_all_loggers():
    """Get a list of all currently active loggers."""
    loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
    return loggers

def remove_console_handlers():
    """Remove console handlers from all active loggers."""
    for logger in get_all_loggers():
        handlers_to_remove = [h for h in logger.handlers if isinstance(h, logging.StreamHandler)]
        for handler in handlers_to_remove:
            logger.removeHandler(handler)

# Example usage
remove_console_handlers()

In [8]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Get a list of all available fonts
available_fonts = fm.findSystemFonts(fontpaths=None, fontext='ttf')

# Print the list of available fonts
for font in available_fonts:
    print(font)

/usr/share/fonts/truetype/noto/NotoSansMalayalam-Bold.ttf
/usr/share/fonts/truetype/noto/NotoSansLisu-Regular.ttf
/usr/share/fonts/truetype/teluguvijayam/NATS.ttf
/usr/share/fonts/opentype/urw-base35/NimbusMonoPS-Regular.otf
/usr/share/fonts/opentype/ebgaramond/EBGaramond12-Bold.otf
/usr/share/fonts/truetype/tlwg/TlwgMono-Oblique.ttf
/usr/share/fonts/opentype/stix-word/STIX-BoldItalic.otf
/usr/share/fonts/opentype/stix/STIXSizeOneSym-Bold.otf
/usr/share/fonts/opentype/urw-base35/URWBookman-DemiItalic.otf
/usr/share/fonts/truetype/noto/NotoSansJavanese-Regular.ttf
/usr/share/fonts/truetype/noto/NotoSansBassaVah-Regular.ttf
/usr/share/fonts/opentype/neohellenic/GFSNeohellenicBold.otf
/usr/share/fonts/truetype/adf/BerenisADFPro-Bold.otf
/usr/share/fonts/opentype/malayalam/Manjari-Bold.otf
/usr/share/fonts/opentype/artemisia/GFSArtemisiaBoldIt.otf
/usr/share/fonts/truetype/malayalam/Suruma.ttf
/usr/share/fonts/truetype/croscore/Arimo-Regular.ttf
/usr/share/fonts/truetype/kacst/KacstDigital

In [7]:
num_processes = int(.75*os.cpu_count())

with multiprocessing.get_context("spawn").Pool(initializer=parallel.initialize_process) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.model_copy(deep=True), visualize_options] 
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-10-18 17:01:09,592 - thor.utils - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/data.yml


KeyboardInterrupt: 

In [4]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-10-18 13:56:10,369 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-10-18 13:56:18,064 - thor.parallel - INFO - Stitching record files.
2024-10-18 13:56:18,112 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/records/filepaths/gridrad.csv
2024-10-18 13:56:18,117 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/records/filepaths/gridrad.yml
2024-10-18 13:56:18,121 - thor.parallel - INFO - Stitching attribute files.
2024-10-18 13:56:18,293 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/attributes/mcs/anvil/core.csv
2024-10-18 13:56:18,297 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/attributes/mcs/anvil/core.yml
2024-10-18 13:56:18,423 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridra

In [5]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-18 13:56:25,533 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.csv
2024-10-18 13:56:25,541 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.yml
2024-10-18 13:56:25,773 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.csv
2024-10-18 13:56:25,777 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.yml
2024-10-18 13:56:25,841 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.csv
2024-10-18 13:56:25,845 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.yml


stratiform_offset inflow  \
time                universal_id                            
2010-01-20 18:00:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:10:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:20:00 1                      leading  right   
...                                            ...    ...   
2010-01-21 03:10:00 20                       right  front   
                    21                     leading  front   
                    23                    trailing  front   
2010-01-21 03:20:00 20                       right  front   
                    21                     leading  front   

                                 relative_stratiform_offset  \
time                universal_id                              
2010-01-20 18:00:00 1                                  left   
                    2                                  left   
2010-01-20 18:10:00 1                                  left   
                    2                                  left   
2010-01-20 18:20:00 1                                  left   
...                                                     ...   
2010-01-21 03:10:00 20                                right   
                    21                              leading   
                    23                             trailing   
2010-01-21 03:20:00 20                                right   
                    21                              leading   

                                                 tilt          propagation  
time                universal_id                                            
2010-01-20 18:00:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:10:00 1                      down-shear           down-shear  
                    2             shear-perpendicular  shear-perpendicular  
2010-01-20 18:20:00 1             shear-perpendicular           down-shear  
...                                               ...                  ...  
2010-01-21 03:10:00 20            shear-perpendicular           down-shear  
                    21                     down-shear           down-shear  
                    23                       up-shear           down-shear  
2010-01-21 03:20:00 20            shear-perpendicular           down-shear  
                    21                     down-shear           down-shear  

[252 rows x 5 columns]

In [3]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2010-01-20T18:00")
end_time = np.datetime64(np.datetime64("2010-01-21T03:30"))
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 5400, "by_date": False}
visualize.attribute.mcs_series(*args, **args_dict)

2024-10-16 21:59:26,974 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-16 21:59:27,157 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-16 21:59:27,862 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-16 21:59:27,928 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-16 21:59:28,355 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-16 21:59:28,604 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-16 21:59:28,965 - thor.data.gridrad - DEBUG - Got domain mask for gridrad.
2024-10-16 21:59:29,063 - thor.visualize.attribute - DEBUG - Getting grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-16 21:59:29,064 - thor.data.gridrad - DEBUG - Got grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-16 21:59:29,065 - thor.visualize.attribute - DEBUG - Rebuilding processed grid for time